# MLP!

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read in all the names
words = open("names.txt", 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
# build the vocab of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


# 11:04

In [7]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  
  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [9]:
X.shape, X.dtype,  Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [54]:
C = torch.randn((27,2))
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [ ]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [75]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h

tensor([[ 0.9800,  0.9978,  0.6881,  ...,  0.1494,  0.0364, -0.3348],
        [ 0.9875,  1.0000,  0.9343,  ..., -1.0000, -0.5982,  0.8723],
        [ 1.0000,  1.0000,  1.0000,  ..., -0.2841, -0.9988, -0.3883],
        ...,
        [ 0.9525,  1.0000,  0.9907,  ..., -0.9999, -0.4526,  0.4484],
        [ 0.9978,  0.9753,  0.9997,  ...,  0.9998, -0.9769, -0.9974],
        [-0.6927, -0.9120, -0.9987,  ...,  0.9999,  0.9853, -0.9657]])

In [76]:
h.shape

torch.Size([32, 100])

In [78]:
W2 = torch.randn((100,27))
b2 = torch.randn((27))

In [79]:
logits = h @ W2 + b2

In [81]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)
prob.shape

torch.Size([32, 27])

In [87]:
prob[torch.arange(32), Y]

tensor([4.1118e-04, 7.6724e-01, 7.5882e-10, 4.5852e-04, 3.6878e-09, 7.9126e-06,
        1.1773e-04, 1.5057e-14, 1.0022e-16, 3.1321e-10, 1.2724e-15, 3.1202e-03,
        1.0873e-05, 1.4051e-08, 3.0006e-12, 1.0141e-01, 6.1823e-08, 7.8406e-07,
        4.5028e-14, 1.3032e-10, 1.5115e-07, 1.9842e-13, 7.9591e-05, 6.7815e-09,
        7.5419e-08, 1.1577e-05, 3.6448e-05, 1.3074e-09, 1.0993e-03, 1.2039e-06,
        5.4687e-13, 3.8367e-07])

In [82]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

## 32:01